In [5]:
import numpy as np
import pandas as pd
import tensorflow as tf
import fasttext
import keras
from underthesea import word_tokenize
from gensim.utils import simple_preprocess

from tensorflow.keras.preprocessing import sequence
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras import layers, Model
from tensorflow.keras.layers import concatenate
from tensorflow.keras.initializers import Constant


from tensorflow.keras.layers import Dense, Dropout, Embedding, LSTM, Bidirectional,Conv1D,GRU,SpatialDropout1D,Concatenate,Input,Flatten,GlobalMaxPooling1D,Reshape,GlobalAveragePooling1D
from tensorflow.keras.callbacks import ReduceLROnPlateau, EarlyStopping
from tensorflow.keras.layers import MaxPooling1D,AveragePooling1D

In [6]:
from numpy import loadtxt
X = loadtxt('X_absa.txt',delimiter=',',dtype = np.int32)
y_aspect = loadtxt('y_aspect.txt',delimiter=',',dtype = np.int32)
loaded_arr = loadtxt("y_polarity.txt",dtype = np.int32,delimiter=',')
y_pol = loaded_arr.reshape(
    loaded_arr.shape[0], loaded_arr.shape[1] //4, 4)
embedding_matrix = loadtxt('embedding_matrix_absa.csv',delimiter=',',dtype = None)
import pickle
a_file = open('wordtoix_ABSA.pkl','rb')
wordtoix = pickle.load(a_file)

a_file = open('ixtoword_ABSA.pkl','rb')
ixtoword = pickle.load(a_file)

In [7]:
max_length  = 382
vocab_size =len(wordtoix)
embedding_dim = 300 

In [10]:
from keras.initializers import Constant
inputs = Input(shape=(max_length,))
embedding = Embedding(
            vocab_size,
            embedding_dim,
            embeddings_initializer=Constant(embedding_matrix))(inputs)
embedding = SpatialDropout1D(0.5)(embedding)
lstm = Bidirectional(LSTM(64,return_sequences=True))(embedding)
conv1d = Conv1D(filters=32, kernel_size=3, padding='same')(lstm)
global_avg_pool = GlobalAveragePooling1D()(conv1d)
global_max_pool  = GlobalMaxPooling1D()(conv1d)
concat_layer = Concatenate(axis = -1)([global_avg_pool,global_avg_pool])

aspect_output = layers.Dense(12, activation="sigmoid",name ='output_aspect')(concat_layer)

output_pol1 = Dense(4,activation = 'softmax',)(concat_layer)
output_pol1 = Reshape((-1,4))(output_pol1)

output_pol2 = Dense(4,activation = 'softmax')(concat_layer)
output_pol2 = Reshape((-1,4))(output_pol2)

output_pol3 = Dense(4,activation = 'softmax')(concat_layer)
output_pol3 = Reshape((-1,4))(output_pol3)

output_pol4 = Dense(4,activation = 'softmax')(concat_layer)
output_pol4 = Reshape((-1,4))(output_pol4)

output_pol5 = Dense(4,activation = 'softmax')(concat_layer)
output_pol5 = Reshape((-1,4))(output_pol5)

output_pol6 = Dense(4,activation = 'softmax')(concat_layer)
output_pol6 = Reshape((-1,4))(output_pol6)

output_pol7 = Dense(4,activation = 'softmax')(concat_layer)
output_pol7 = Reshape((-1,4))(output_pol7)

output_pol8 = Dense(4,activation = 'softmax')(concat_layer)
output_pol8 = Reshape((-1,4))(output_pol8)

output_pol9 = Dense(4,activation = 'softmax')(concat_layer)
output_pol9 = Reshape((-1,4))(output_pol9)

output_pol10 = Dense(4,activation = 'softmax')(concat_layer)
output_pol10 = Reshape((-1,4))(output_pol10)

output_pol11 = Dense(4,activation = 'softmax')(concat_layer)
output_pol11 = Reshape((-1,4))(output_pol11)

output_pol12 = Dense(4,activation = 'softmax')(concat_layer)
output_pol12 = Reshape((-1,4))(output_pol12)

output_pol = concatenate([output_pol1,output_pol2,output_pol3,output_pol4,output_pol5,
                    output_pol6,output_pol7,output_pol8,output_pol9,output_pol10,output_pol11,output_pol12],name='output_polarity',axis = 1)

model3 = Model(inputs = inputs ,outputs =[aspect_output,output_pol])

In [11]:
model3.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_3 (InputLayer)           [(None, 382)]        0           []                               
                                                                                                  
 embedding_2 (Embedding)        (None, 382, 300)     1212000     ['input_3[0][0]']                
                                                                                                  
 spatial_dropout1d_1 (SpatialDr  (None, 382, 300)    0           ['embedding_2[0][0]']            
 opout1D)                                                                                         
                                                                                                  
 bidirectional_1 (Bidirectional  (None, 382, 128)    186880      ['spatial_dropout1d_1[0][0]']

                                                                  'reshape_10[0][0]',             
                                                                  'reshape_11[0][0]']             
                                                                                                  
Total params: 1,415,100
Trainable params: 1,415,100
Non-trainable params: 0
__________________________________________________________________________________________________


In [12]:
callback = tf.keras.callbacks.EarlyStopping(patience=5, restore_best_weights=True)

In [13]:
loss = {'output_aspect' : "binary_crossentropy" , 'output_polarity': 'categorical_crossentropy'}
model3.compile(optimizer='adam', loss=loss,loss_weights=[0.1,1])

In [14]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_aspect_train,y_aspect_test,y_pol_train,y_pol_test = train_test_split(X, y_aspect,y_pol,test_size= 0.22 ,random_state=42)

In [ ]:
history = model3.fit(X_train,[y_aspect_train,y_pol_train],
                   batch_size=64,
                    epochs=60,
                    validation_data=[X_test, [y_aspect_test,y_pol_test]],callbacks=[callback])
model3.save('lstm_model_absa.hdf5')

In [24]:
model  = load_model('lstm_model_absa.hdf5')

In [ ]:
import matplotlib.pyplot as plt
plt.plot(history.history['loss'], label='loss')
plt.plot(history.history['val_loss'], label='val_loss')
plt.xlabel('epoch')
plt.legend()
plt.show() 

In [ ]:
def predict_aspect_pol(text):
    token_sample = word_processing(text)
    X_predict=[wordtoix[word] for word in token_sample]
    X_predict=sequence.pad_sequences([X_predict],max_length)
    predict= model.predict(X_predict)
    
    i = 0
    label_aspect_pred_lst= []
    for pred in predict[0][0]:
        if pred > 0.0002 :
    #         print(label_aspect_rev[i])
            label_aspect_pred_lst.append(label_aspect_rev[i])
        i = i+1
        
    j =0 
    label_pol_aspect_dict ={}
    for pre_pol in predict[1][0]:
        if pre_pol.argmax() != 0:
    #         print(label_aspect_rev[i],pre_pol.argmax())
            label_pol_aspect_dict[label_aspect_rev[j]]= label_polarity_rev[pre_pol.argmax()]

        j = j +1
    label_aspect_final = {}
    for label in label_pol_aspect_dict.keys():
        if label in label_aspect_pred_lst:
#             print(label,label_pol_aspect_dict[label])
             label_aspect_final[label] = label_pol_aspect_dict[label]
    return label_aspect_final

In [ ]:
text = 'phục vụ tận tình'
predict_aspect_pol(text)